# Combine the analysis tiles

And mask regions such as urban, agriculture, and water bodies.

In [ ]:
%matplotlib inline

import os
import sys
import warnings
import numpy as np
import xarray as xr
import pandas as pd
from odc.geo.xr import assign_crs
import matplotlib.pyplot as plt

sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import round_coords


In [ ]:
data_type = 'MODIS_GPP'  #'AusEFlux_GPP' 'DIFFUSE_GPP' 'NDVI_1982_2022' 'MODIS_GPP'
tiles_path = f'/g/data/os22/chad_tmp/Aus_CO2_fertilisation/results/tiles/{data_type}/'
export=f'/g/data/os22/chad_tmp/Aus_CO2_fertilisation/results/combined_tiles/{data_type}/'

## Open masking datasets

In [ ]:
crops = xr.open_dataset('/g/data/os22/chad_tmp/Aus_phenology/data/croplands_5km.nc')['croplands']
crops = xr.where(crops==1, 0, 1) #irrigated crops
crops = round_coords(crops)

urban = xr.open_dataarray('/g/data/xc0/project/AusEFlux/data/urban_water_mask_5km.nc')
mask = (urban | crops)

mask = ~mask.astype('bool')

## Join tiles together

and mask with urban etc.

In [ ]:
def combine_tiles(path, v, mask):
    tiles = [path+i for i in os.listdir(path) if v in i]
    tiles = [xr.open_dataset(t) for t in tiles]
    p = xr.combine_by_coords(tiles)
    p = assign_crs(p,crs='EPSG:4326')
    
    for var in p.data_vars:
        try:
            del p[var].attrs['grid_mapping']
        except:
            continue

    # mask = (urban & crops)
    p = p.where(mask)
    return p

p_trends = combine_tiles(tiles_path, 'trends', mask)
# p_beta = combine_tiles(tiles_path, 'beta', mask)
# p_attribute_pls = combine_tiles(tiles_path, 'PLS', mask)
# p_attribute_ml = combine_tiles(tiles_path, 'ML', mask)
# p_attribute_delta_slope = combine_tiles(tiles_path, 'delta_slope', mask)


## Interactive maps

In [ ]:
p_trends['slope'].odc.explore( #.where(p_trends['p_value']<=0.1)
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr = 'Esri',
    name = 'Esri Satellite',
    cmap='BrBG',
    robust=True
)

In [ ]:
# fig,ax=plt.subplots(1,2, figsize=(10,4), sharey=True)
# p_beta['beta'].where(p_beta['pvalue']<=0.1).plot(ax=ax[0], vmin=0, vmax=0.025, add_labels=False, cmap='magma')
# (p_beta['beta_relative']).where(p_beta['pvalue']<=0.1).plot(ax=ax[1], vmin=0, vmax=150, add_labels=False, cmap='magma')
# ax[0].set_title('Beta NDVI / yr / ppm')
# ax[1].set_title('Beta relative NDVI / 100ppm');

In [ ]:
# p_beta['beta_relative'].where(p_beta['pvalue']<=0.05).odc.explore( #.where(p_trends['p_value']<=0.1)
#     tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#     attr = 'Esri',
#     name = 'Esri Satellite',
#     cmap='magma',
#     robust=True
# )

## Export

In [ ]:
p_trends.to_netcdf(f'{export}/trends_{data_type}.nc')
# p_beta.to_netcdf(f'{export}beta_{data_type}.nc')
# p_attribute_pls.to_netcdf(f'{export}attribution_PLS_{data_type}.nc')
# p_attribute_ml.to_netcdf(f'{export}attribution_ML_{data_type}.nc')
# p_attribute_delta_slope.to_netcdf(f'{export}/attribution_delta_slope_{data_type}.nc')